# Fraud Detection - Multiclass Classification
## Machine Learning I - Group Project

**Objective:** Develop a machine learning system for e-commerce fraud detection using multiclass classification (Legitimate, Suspicious, Fraudulent).

**Authors:** Group C 
**Date:** December 2024

---

### Project Structure:
- **4 ML Algorithms:** ANNs, SVMs, Decision Trees, kNN
- **Ensemble Methods:** Majority Voting, Weighted Voting
- **Dataset:** 1.5M e-commerce transactions
- **Target:** 3-class risk assessment (0=Legit, 1=Suspect, 2=Fraud)

In [1]:
# ============================================================================
#                    SETUP & PACKAGE IMPORTS
# ============================================================================

# Set random seed for reproducibility
using Random
Random.seed!(42)

# Load packages
using CSV
using DataFrames
using Flux
using Statistics
using Dates
using StatsBase
using LinearAlgebra
using LIBSVM
using DecisionTree
using NearestNeighbors

println("✅ Packages loaded!")

# Load custom modules from utils/
include("utils/preprocessing.jl")
include("utils/metrics.jl")
include("utils/ann_utils.jl")
include("utils/svm_utils.jl")
include("utils/tree_utils.jl")
include("utils/knn_utils.jl")
include("utils/ensemble_utils.jl")

using .PreprocessingUtils
using .MetricsUtils
using .ANNUtils
using .SVMUtils
using .TreeUtils
using .KNNUtils
using .EnsembleUtils

println("✅ Custom modules loaded!")

✅ Packages loaded!
✅ Custom modules loaded!


## 1. Data Loading & Exploration

Loading the **Fraudulent E-Commerce Transaction Dataset** containing:
- 1.5M transactions
- 16 original features
- Binary fraud labels (will be converted to 3-class)

**Dataset Source:** Public e-commerce fraud dataset  
**Original Size:** 1,496,586 transactions

In [2]:
# ============================================================================
#                    DATA LOADING
# ============================================================================

const DATA_PATH = "datasets/Fraudulent_E-Commerce_Transaction_Data_merge.csv"
println("\n" * "="^70)
println("📂 LOADING DATA")
println("="^70)
println("\nLoading dataset from: $DATA_PATH")

df = CSV.read(DATA_PATH, DataFrame)
target_col = "Is Fraudulent"

println("Original dataset size: $(size(df))")
println("Columns: $(names(df))")


📂 LOADING DATA

Loading dataset from: datasets/Fraudulent_E-Commerce_Transaction_Data_merge.csv
Original dataset size: (1496586, 16)
Columns: ["Transaction ID", "Customer ID", "Transaction Amount", "Transaction Date", "Payment Method", "Product Category", "Quantity", "Customer Age", "Customer Location", "Device Used", "IP Address", "Shipping Address", "Billing Address", "Is Fraudulent", "Account Age Days", "Transaction Hour"]


## 2. Target Creation: 3-Class Risk Assessment

**Converting binary fraud labels to 3-class risk levels:**

### Risk Signals Used:
1. **Time Risk:** Transactions at unusual hours (0-5am, 11pm)
2. **Amount Risk:** Transactions above 90th percentile
3. **Account Age Risk:** New accounts (<30 days)

### Class Definition:
- **Class 0 (LEGITTIMO):** Non-fraud + low risk signals
- **Class 1 (SOSPETTO):** Borderline cases requiring manual review
  - Frauds with few risk signals
  - Non-frauds with multiple risk signals
- **Class 2 (FRAUDOLENTO):** Clear fraud with multiple risk signals

**Strategy:** Total_Risk = Hour_Risk + Amount_Risk + Account_Risk

In [3]:
# ============================================================================
#                3-CLASS TARGET CREATION
# ============================================================================

println("\n" * "="^70)
println("🎯 CREATING 3-CLASS TARGET")
println("="^70)

df_with_classes = create_risk_classes(df, target_col)


🎯 CREATING 3-CLASS TARGET

 Calculating risk signals...

✅ 3-Class distribution:
  Class 0 (LEGITTIMO): 1347548 (90.0%)
  Class 1 (SOSPETTO): 128384 (8.6%)
  Class 2 (FRAUDOLENTO): 20654 (1.4%)


Row,Transaction ID,Customer ID,Transaction Amount,Transaction Date,Payment Method,Product Category,Quantity,Customer Age,Customer Location,Device Used,IP Address,Shipping Address,Billing Address,Is Fraudulent,Account Age Days,Transaction Hour,Hour_Risk,Amount_Risk,Account_Risk,Total_Risk,Risk_Class
,String,String,Float64,String31,String15,String15,Int64,Int64,String31,String7,String15,String,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,15d2e414-8735-46fc-9e02-80b472b2580f,d1b87f62-51b2-493b-ad6a-77e0fe13e785,58.09,2024-02-20 05:58:41,bank transfer,electronics,1,17,Amandaborough,tablet,212.195.49.198,Unit 8934 Box 0058\nDPO AA 05437,Unit 8934 Box 0058\nDPO AA 05437,0,30,5,1,0,0,1,0
2,0bfee1a0-6d5e-40da-a446-d04e73b1b177,37de64d5-e901-4a56-9ea0-af0c24c069cf,389.96,2024-02-25 08:09:45,debit card,electronics,2,40,East Timothy,desktop,208.106.249.121,"634 May Keys\nPort Cherylview, NV 75063","634 May Keys\nPort Cherylview, NV 75063",0,72,8,0,0,0,0,0
3,e588eef4-b754-468e-9d90-d0e0abfc1af0,1bac88d6-4b22-409a-a06b-425119c57225,134.19,2024-03-18 03:42:55,PayPal,home & garden,2,22,Davismouth,tablet,76.63.88.212,"16282 Dana Falls Suite 790\nRothhaven, IL 15564","16282 Dana Falls Suite 790\nRothhaven, IL 15564",0,63,3,1,0,0,1,0
4,4de46e52-60c3-49d9-be39-636681009789,2357c76e-9253-4ceb-b44e-ef4b71cb7d4d,226.17,2024-03-16 20:41:31,bank transfer,clothing,5,31,Lynnberg,desktop,207.208.171.73,"828 Strong Loaf Apt. 646\nNew Joshua, UT 84798","828 Strong Loaf Apt. 646\nNew Joshua, UT 84798",0,124,20,0,0,0,0,0
5,074a76de-fe2d-443e-a00c-f044cdb68e21,45071bc5-9588-43ea-8093-023caec8ea1c,121.53,2024-01-15 05:08:17,bank transfer,clothing,2,51,South Nicole,tablet,190.172.14.169,"29799 Jason Hills Apt. 439\nWest Richardtown, OH 36093","29799 Jason Hills Apt. 439\nWest Richardtown, OH 36093",0,158,5,1,0,0,1,0
6,4e707452-7c8a-4cbd-b0c1-2aeaa35c5e88,29616b04-2d5c-4729-9c9d-8d71a6ad9dc1,166.41,2024-01-30 10:55:14,bank transfer,toys & games,2,34,Herreramouth,tablet,202.237.29.55,"5699 Brittany Villages Suite 903\nLake Tim, MD 46274","120 Kristi Dale\nPort Meganshire, GU 03060",0,38,10,0,0,0,0,0
7,7ed952fe-8ae1-4f11-8cc5-6607060240d8,fe21ae29-ba4c-424f-9d55-0095539c09fa,92.88,2024-02-04 19:59:10,PayPal,toys & games,2,14,Ramosfort,tablet,13.45.27.192,"727 Gibson Islands Apt. 279\nNew Davidbury, ME 43104","727 Gibson Islands Apt. 279\nNew Davidbury, ME 43104",0,119,19,0,0,0,0,0
8,0b2fb5aa-7171-472f-8269-371094608a07,024257c3-5671-4de8-a33c-98fc5cbe6f92,318.14,2024-02-20 13:30:29,credit card,health & beauty,4,42,Port Emily,desktop,131.141.230.185,"3914 Davis Union\nBrownchester, IN 07744","3914 Davis Union\nBrownchester, IN 07744",0,251,13,0,0,0,0,0
9,1f52366c-7f40-4397-885f-3856b6e6531c,f17640ca-49da-45d1-8461-c2a1cf9c1b61,47.92,2024-03-03 19:44:00,bank transfer,home & garden,4,38,Carneyfurt,desktop,210.148.17.240,"47893 Maldonado Stream Suite 443\nBrownshire, MO 48487","47893 Maldonado Stream Suite 443\nBrownshire, MO 48487",0,190,19,0,0,0,0,0


## 3. Train/Test Split & Class Balancing

**Data Preparation Strategy:**

### Balancing:
- Original distribution: 90% Legit, 8.6% Suspect, 1.4% Fraud (**highly imbalanced**)
- Solution: **Undersample** to equal class sizes (20,654 samples per class)
- Total balanced dataset: **61,962 samples**

### Train/Test Split:
- **80% Training** (49,569 samples) - for model selection via cross-validation
- **20% Test** (12,393 samples) - for final evaluation (never seen during training)

**Critical:** Test set is held out completely until final evaluation to avoid data leakage!

In [4]:
# ============================================================================
#            CLASS BALANCING & TRAIN/TEST SPLIT
# ============================================================================

println("\n" * "="^70)
println("✅ TRAIN/TEST SPLIT (80% Train / 20% Test)")
println("="^70)

println("\n🔄 Balancing 3-class dataset...")

class_0 = df_with_classes[df_with_classes.Risk_Class .== 0, :]
class_1 = df_with_classes[df_with_classes.Risk_Class .== 1, :]
class_2 = df_with_classes[df_with_classes.Risk_Class .== 2, :]

n_min = minimum([size(class_0, 1), size(class_1, 1), size(class_2, 1)])
n_target = min(n_min, 40000)

println("  Samples per class: $n_target")

class_0_sample = class_0[shuffle(1:size(class_0, 1))[1:n_target], :]
class_1_sample = class_1[shuffle(1:size(class_1, 1))[1:n_target], :]
class_2_sample = class_2[shuffle(1:size(class_2, 1))[1:n_target], :]

df_balanced = vcat(class_0_sample, class_1_sample, class_2_sample)
df_balanced = df_balanced[shuffle(1:size(df_balanced, 1)), :]

println("  Balanced dataset size: $(size(df_balanced))")

# Split Train/Test BEFORE preprocessing (critical!)
n_total = size(df_balanced, 1)
n_train = floor(Int, n_total * 0.80)
n_test = n_total - n_train

all_indices = shuffle(1:n_total)
train_indices = all_indices[1:n_train]
test_indices = all_indices[n_train+1:end]

df_train = df_balanced[train_indices, :]
df_test = df_balanced[test_indices, :]

println("\n📊 Split Summary:")
println("  Total samples:     $n_total")
println("  Training set:      $n_train (80%)")
println("  Test set:          $n_test (20%)")


✅ TRAIN/TEST SPLIT (80% Train / 20% Test)

🔄 Balancing 3-class dataset...
  Samples per class: 20654
  Balanced dataset size: (61962, 21)

📊 Split Summary:
  Total samples:     61962
  Training set:      49569 (80%)
  Test set:          12393 (20%)


## 4. Preprocessing & Feature Engineering

**Preprocessing Steps:**

### 1. Time Features:
- Extract hour from transaction timestamp
- Create `Is_Night` flag (hour < 6)

### 2. Feature Engineering:
- `Amount_per_AccountAge`: Transaction amount relative to account age
- `High_Value_Flag`: Transactions > 95th percentile
- `New_Account_Flag`: Accounts < 30 days old

### 3. Missing Value Imputation:
- Replace missing values with median

### 4. Feature Selection:
- Drop: IDs, addresses, dates, categorical features
- **Final Features:** 8 numerical features

### 5. Normalization:
- **Method:** Min-Max normalization [0, 1]
- **Parameters:** Calculated from training set ONLY
- **Critical:** Test set normalized using training parameters (no data leakage!)

In [5]:
# ============================================================================
#                    PREPROCESSING
# ============================================================================

println("\n🔧 Preprocessing train and test sets...")
df_train_processed = preprocess_multiclass(df_train, target_col)
df_test_processed = preprocess_multiclass(df_test, target_col)

input_cols = setdiff(names(df_train_processed), ["Risk_Class"])
train_inputs = Matrix{Float32}(df_train_processed[:, input_cols])
train_targets = Int.(df_train_processed.Risk_Class)

test_inputs = Matrix{Float32}(df_test_processed[:, input_cols])
test_targets = Int.(df_test_processed.Risk_Class)

println("\n📊 Preprocessed Data:")
println("  Features: $(length(input_cols))")
println("  Train samples: $(size(train_inputs, 1))")
println("  Test samples: $(size(test_inputs, 1))")
println("\nFeature names: $input_cols")


🔧 Preprocessing train and test sets...

📊 Preprocessed Data:
  Features: 8
  Train samples: 49569
  Test samples: 12393

Feature names: ["Transaction Amount", "Account Age Days", "Transaction Hour", "Transaction_Hour", "Is_Night", "Amount_per_AccountAge", "High_Value_Flag", "New_Account_Flag"]


# Experiment 1: Artificial Neural Networks (ANNs)

## Overview:
- **Tested:** 8 different architectures
- **Layers:** 1-2 hidden layers with varying neurons
- **Activation:** ReLU for hidden layers, Softmax for output
- **Loss:** Weighted Cross-Entropy (weights: [1.0, 2.0, 3.0] for [Legit, Suspect, Fraud])
- **Optimizer:** Adam (learning rate: 0.003)
- **Regularization:** Early stopping with validation set (10% of training)

## Architectures Tested:
1. [256, 128, 64] - Deep Large (3 layers)
2. [128, 64, 32] - Baseline (3 layers)
3. [96, 48, 24] - Medium (3 layers)
4. [64, 32] - Shallow (2 layers)
5. [128, 128, 64, 32] - Very Deep (4 layers)
6. [192, 96, 48] - Wide-Deep (3 layers)
7. [128, 64] - Simple (2 layers)
8. [256, 128] - Large (2 layers)

## Methodology:
- **3-Fold Stratified Cross-Validation** on training set
- Select best architecture based on Macro F1 score
- Retrain on full training set
- Evaluate once on test set

In [6]:
# ============================================================================
#        EXPERIMENT 1: ARTIFICIAL NEURAL NETWORKS (ANNs)
# ============================================================================

println("\n" * "="^70)
println("🔬 EXPERIMENT 1: ARTIFICIAL NEURAL NETWORKS")
println("Testing 8 ANN Topologies")
println("="^70)

topologies_to_test = [
    [256, 128, 64],      # 1. Deep Large
    [128, 64, 32],       # 2. Baseline
    [96, 48, 24],        # 3. Medium
    [64, 32],            # 4. Shallow
    [128, 128, 64, 32],  # 5. Very Deep
    [192, 96, 48],       # 6. Wide-Deep
    [128, 64],           # 7. Simple 2-Layer
    [256, 128]           # 8. Large 2-Layer
]

learningRate_ann = 0.003
maxEpochs_ann = 800
maxEpochsVal_ann = 25
k_folds = 3
classWeights_ann = [1.0, 2.0, 3.0]

println("\n⚙️  Configuration:")
println("  Learning Rate: $learningRate_ann")
println("  Max Epochs: $maxEpochs_ann")
println("  Early Stopping Patience: $maxEpochsVal_ann epochs")
println("  K-Folds: $k_folds")
println("  Class Weights: $classWeights_ann")

cv_indices = crossvalidation_multiclass(train_targets, k_folds)

topology_results = []

for (i, topology) in enumerate(topologies_to_test)
    println("\n[$i/8] Testing topology: $topology")
    
    macro_f1, weighted_f1, cm, class_metrics = multiclassANNCrossValidation(
        topology,
        (train_inputs, train_targets),
        cv_indices;
        maxEpochs = maxEpochs_ann,
        learningRate = learningRate_ann,
        maxEpochsVal = maxEpochsVal_ann,
        classWeights = classWeights_ann,
        verbose = true
    )
    
    push!(topology_results, (topology, macro_f1, weighted_f1, cm, class_metrics))
end

sorted_ann_results = sort(topology_results, by=x->x[2], rev=true)

println("\n🏆 ANN Results Ranking:")
println("-"^70)
for (i, (topo, macro_f1, weighted_f1, cm, class_metrics)) in enumerate(sorted_ann_results)
    badge = i == 1 ? "🥇" : i == 2 ? "🥈" : i == 3 ? "🥉" : "  "
    println("$badge $i. $topo - Macro F1: $(round(macro_f1*100, digits=2))%")
end

best_topology_ann, best_macro_f1_ann, best_weighted_f1_ann = sorted_ann_results[1][1:3]

println("\n✨ Best ANN: $best_topology_ann")
println("   CV Macro F1: $(round(best_macro_f1_ann*100, digits=2))%")


🔬 EXPERIMENT 1: ARTIFICIAL NEURAL NETWORKS
Testing 8 ANN Topologies

⚙️  Configuration:
  Learning Rate: 0.003
  Max Epochs: 800
  Early Stopping Patience: 25 epochs
  K-Folds: 3
  Class Weights: [1.0, 2.0, 3.0]

[1/8] Testing topology: [256, 128, 64]

  Testing topology: [256, 128, 64]
    → Macro F1: 77.14% | Weighted F1: 77.12%

[2/8] Testing topology: [128, 64, 32]

  Testing topology: [128, 64, 32]
    → Macro F1: 77.05% | Weighted F1: 77.04%

[3/8] Testing topology: [96, 48, 24]

  Testing topology: [96, 48, 24]
    → Macro F1: 77.16% | Weighted F1: 77.14%

[4/8] Testing topology: [64, 32]

  Testing topology: [64, 32]
    → Macro F1: 77.16% | Weighted F1: 77.15%

[5/8] Testing topology: [128, 128, 64, 32]

  Testing topology: [128, 128, 64, 32]
    → Macro F1: 77.04% | Weighted F1: 77.02%

[6/8] Testing topology: [192, 96, 48]

  Testing topology: [192, 96, 48]
    → Macro F1: 76.88% | Weighted F1: 76.86%

[7/8] Testing topology: [128, 64]

  Testing topology: [128, 64]
    → M

In [7]:
# Train final ANN on full train set
println("\n🚀 Training final ANN on full training set...")

train_targets_onehot = oneHotEncoding(train_targets, [0, 1, 2])
n_train_full = size(train_inputs, 1)
n_val_final = floor(Int, n_train_full * 0.10)
val_final_indices = shuffle(1:n_train_full)[1:n_val_final]
train_final_indices = setdiff(1:n_train_full, val_final_indices)

train_final_inputs = train_inputs[train_final_indices, :]
train_final_targets = train_targets_onehot[train_final_indices, :]
val_final_inputs = train_inputs[val_final_indices, :]
val_final_targets = train_targets_onehot[val_final_indices, :]

normParams_ann = calculateMinMaxNormalizationParameters(train_final_inputs)
train_final_inputs_norm_ann = normalizeMinMax(train_final_inputs, normParams_ann)
val_final_inputs_norm_ann = normalizeMinMax(val_final_inputs, normParams_ann)

final_ann = trainMulticlassANN(
    best_topology_ann,
    (train_final_inputs_norm_ann, Bool.(train_final_targets));
    validationDataset = (val_final_inputs_norm_ann, Bool.(val_final_targets)),
    maxEpochs = maxEpochs_ann,
    learningRate = learningRate_ann,
    maxEpochsVal = maxEpochsVal_ann,
    classWeights = classWeights_ann
)

println("✅ Final ANN trained!")


🚀 Training final ANN on full training set...
✅ Final ANN trained!


In [8]:
# Evaluate on test set (once!)
println("\n📊 Evaluating ANN on Test Set...")

test_inputs_norm_ann = normalizeMinMax(Matrix{Float32}(test_inputs), normParams_ann)
ann_test_outputs = final_ann(test_inputs_norm_ann')'
ann_test_predictions = [argmax(ann_test_outputs[i, :]) - 1 for i in 1:size(ann_test_outputs, 1)]

cm_ann, acc_ann, class_metrics_ann, macro_f1_ann, weighted_f1_ann = 
    confusionMatrixMulticlass(ann_test_predictions, test_targets, 3)

println("\n📊 ANN TEST RESULTS:")
displayConfusionMatrix(cm_ann)
println("\nOverall Accuracy:  $(round(acc_ann*100, digits=2))%")
println("Macro F1:          $(round(macro_f1_ann*100, digits=2))%")
println("Weighted F1:       $(round(weighted_f1_ann*100, digits=2))%")

println("\nPer-Class Performance:")
for (name, recall, prec, f1) in class_metrics_ann
    println("  $(rpad(name, 12)): Recall=$(round(recall*100, digits=1))% | Precision=$(round(prec*100, digits=1))% | F1=$(round(f1*100, digits=1))%")
end


📊 Evaluating ANN on Test Set...

📊 ANN TEST RESULTS:

📊 CONFUSION MATRIX (3x3):
                  Predicted
              Legit    Sospect   Fraud
----------------------------------------------------------------------
Legittimo    |    3787      360        0
Sospetto     |     583     2227     1282
Fraudolento  |       0      399     3755

Overall Accuracy:  78.83%
Macro F1:          77.86%
Weighted F1:       77.92%

Per-Class Performance:
  Legittimo   : Recall=91.3% | Precision=86.7% | F1=88.9%
  Sospetto    : Recall=54.4% | Precision=74.6% | F1=62.9%
  Fraudolento : Recall=90.4% | Precision=74.5% | F1=81.7%


# Experiment 2: Support Vector Machines (SVMs)

## Overview:
- **Tested:** 10 different configurations
- **Kernels:** Linear, RBF, Polynomial
- **Hyperparameter C:** 0.1, 1.0, 10.0
- **Gamma (RBF):** auto, 0.1
- **Degree (Poly):** 2, 3

## Configurations Tested:
1. Linear C=0.1
2. Linear C=1.0
3. Linear C=10.0
4. RBF C=0.1, γ=auto
5. RBF C=1.0, γ=auto
6. RBF C=10.0, γ=auto
7. RBF C=1.0, γ=0.1
8. Polynomial C=1.0, degree=2
9. Polynomial C=1.0, degree=3
10. Polynomial C=10.0, degree=2

## Methodology:
- **3-Fold Stratified Cross-Validation** on training set
- Select best config based on Macro F1 score
- Retrain on full training set
- Evaluate once on test set

In [9]:
# ============================================================================
#        EXPERIMENT 2: SUPPORT VECTOR MACHINES (SVMs)
# ============================================================================

println("\n" * "="^70)
println("🔬 EXPERIMENT 2: SUPPORT VECTOR MACHINES")
println("Testing 10 SVM Configurations")
println("="^70)

svm_configs = [
    (:linear, 0.1, :auto, 3, "Linear C=0.1"),
    (:linear, 1.0, :auto, 3, "Linear C=1.0"),
    (:linear, 10.0, :auto, 3, "Linear C=10.0"),
    (:rbf, 0.1, :auto, 3, "RBF C=0.1 γ=auto"),
    (:rbf, 1.0, :auto, 3, "RBF C=1.0 γ=auto"),
    (:rbf, 10.0, :auto, 3, "RBF C=10.0 γ=auto"),
    (:rbf, 1.0, 0.1, 3, "RBF C=1.0 γ=0.1"),
    (:polynomial, 1.0, :auto, 2, "Poly C=1.0 deg=2"),
    (:polynomial, 1.0, :auto, 3, "Poly C=1.0 deg=3"),
    (:polynomial, 10.0, :auto, 2, "Poly C=10.0 deg=2")
]

svm_results = []

for (i, (kernel, C, gamma, degree, desc)) in enumerate(svm_configs)
    println("\n[$i/10] Testing: $desc")
    
    macro_f1, weighted_f1, cm, class_metrics = svmCrossValidation(
        (Matrix{Float64}(train_inputs), train_targets),
        cv_indices;
        kernel = kernel,
        C = C,
        gamma = gamma,
        degree = degree,
        verbose = true
    )
    
    push!(svm_results, (desc, kernel, C, gamma, degree, macro_f1, weighted_f1, cm, class_metrics))
end

sorted_svm_results = sort(svm_results, by=x->x[6], rev=true)

println("\n🏆 SVM Results Ranking:")
println("-"^70)
for (i, result) in enumerate(sorted_svm_results)
    badge = i == 1 ? "🥇" : i == 2 ? "🥈" : i == 3 ? "🥉" : "  "
    println("$badge $i. $(result[1]) - Macro F1: $(round(result[6]*100, digits=2))%")
end

best_svm = sorted_svm_results[1]
best_desc_svm, best_kernel_svm, best_C_svm, best_gamma_svm, best_degree_svm, best_macro_f1_svm = best_svm[1:6]

println("\n✨ Best SVM: $best_desc_svm")
println("   CV Macro F1: $(round(best_macro_f1_svm*100, digits=2))%")


🔬 EXPERIMENT 2: SUPPORT VECTOR MACHINES
Testing 10 SVM Configurations

[1/10] Testing: Linear C=0.1
  Testing SVM: kernel=linear, C=0.1, gamma=auto, degree=3
    → Macro F1: 63.22% | Weighted F1: 63.2%

[2/10] Testing: Linear C=1.0
  Testing SVM: kernel=linear, C=1.0, gamma=auto, degree=3
    → Macro F1: 74.19% | Weighted F1: 74.17%

[3/10] Testing: Linear C=10.0
  Testing SVM: kernel=linear, C=10.0, gamma=auto, degree=3
    → Macro F1: 74.64% | Weighted F1: 74.62%

[4/10] Testing: RBF C=0.1 γ=auto
  Testing SVM: kernel=rbf, C=0.1, gamma=auto, degree=3
    → Macro F1: 63.09% | Weighted F1: 63.07%

[5/10] Testing: RBF C=1.0 γ=auto
  Testing SVM: kernel=rbf, C=1.0, gamma=auto, degree=3
    → Macro F1: 68.67% | Weighted F1: 68.65%

[6/10] Testing: RBF C=10.0 γ=auto
  Testing SVM: kernel=rbf, C=10.0, gamma=auto, degree=3
    → Macro F1: 74.85% | Weighted F1: 74.83%

[7/10] Testing: RBF C=1.0 γ=0.1
  Testing SVM: kernel=rbf, C=1.0, gamma=0.1, degree=3
    → Macro F1: 66.19% | Weighted F1: 

In [10]:
# Train final SVM
println("\n🚀 Training final SVM on full training set...")

normParams_svm = calculateMinMaxNormalizationParameters(Matrix{Float64}(train_inputs))
train_inputs_norm_svm = normalizeMinMax(Matrix{Float64}(train_inputs), normParams_svm)
train_targets_float_svm = Float64.(train_targets)

if best_kernel_svm == :rbf
    gamma_final_svm = best_gamma_svm == :auto ? 1.0 / size(train_inputs_norm_svm, 2) : best_gamma_svm
else
    gamma_final_svm = 0.0
end

if best_kernel_svm == :linear
    final_svm = svmtrain(train_inputs_norm_svm', train_targets_float_svm,
                        kernel=LIBSVM.Kernel.Linear, cost=best_C_svm, verbose=false)
elseif best_kernel_svm == :rbf
    final_svm = svmtrain(train_inputs_norm_svm', train_targets_float_svm,
                        kernel=LIBSVM.Kernel.RadialBasis, cost=best_C_svm, gamma=gamma_final_svm, verbose=false)
elseif best_kernel_svm == :polynomial
    final_svm = svmtrain(train_inputs_norm_svm', train_targets_float_svm,
                        kernel=LIBSVM.Kernel.Polynomial, cost=best_C_svm, degree=best_degree_svm, verbose=false)
end

# Evaluate on test set
test_inputs_norm_svm = normalizeMinMax(Matrix{Float64}(test_inputs), normParams_svm)
svm_test_predictions, _ = svmpredict(final_svm, test_inputs_norm_svm')
svm_test_predictions = Int.(svm_test_predictions)

cm_svm, acc_svm, class_metrics_svm, macro_f1_svm, weighted_f1_svm = 
    confusionMatrixMulticlass(svm_test_predictions, test_targets, 3)

println("\n📊 SVM TEST RESULTS:")
displayConfusionMatrix(cm_svm)
println("\nMacro F1: $(round(macro_f1_svm*100, digits=2))% | Accuracy: $(round(acc_svm*100, digits=2))%")


🚀 Training final SVM on full training set...

📊 SVM TEST RESULTS:

📊 CONFUSION MATRIX (3x3):
                  Predicted
              Legit    Sospect   Fraud
----------------------------------------------------------------------
Legittimo    |    3773      374        0
Sospetto     |     732     2080     1280
Fraudolento  |      24      570     3560

Macro F1: 74.87% | Accuracy: 75.95%


# Experiment 3: Decision Trees

## Overview:
- **Tested:** 7 different maximum depths
- **Criterion:** Gini impurity
- **Min samples split:** 2

## Configurations Tested:
1. Depth = 3 (shallow)
2. Depth = 5
3. Depth = 7
4. Depth = 10
5. Depth = 15
6. Depth = 20 (deep)
7. Depth = Unlimited (full tree)

## Advantages:
- Interpretable (can visualize decision rules)
- No need for feature scaling
- Fast training and prediction

In [11]:
# ============================================================================
#            EXPERIMENT 3: DECISION TREES
# ============================================================================

println("\n" * "="^70)
println("🔬 EXPERIMENT 3: DECISION TREES")
println("Testing 7 Configurations")
println("="^70)

tree_configs = [
    (3, "Depth=3"),
    (5, "Depth=5"),
    (7, "Depth=7"),
    (10, "Depth=10"),
    (15, "Depth=15"),
    (20, "Depth=20"),
    (-1, "Depth=Unlimited")
]

tree_results = []

for (i, (max_depth, desc)) in enumerate(tree_configs)
    println("\n[$i/7] Testing: $desc")
    
    macro_f1, weighted_f1, cm, class_metrics = decisionTreeCrossValidation(
        (Matrix{Float64}(train_inputs), train_targets),
        cv_indices;
        max_depth = max_depth,
        min_samples_split = 2,
        verbose = true
    )
    
    push!(tree_results, (desc, max_depth, macro_f1, weighted_f1, cm, class_metrics))
end

sorted_tree_results = sort(tree_results, by=x->x[3], rev=true)

println("\n🏆 Decision Tree Results Ranking:")
println("-"^70)
for (i, result) in enumerate(sorted_tree_results)
    badge = i == 1 ? "🥇" : i == 2 ? "🥈" : i == 3 ? "🥉" : "  "
    println("$badge $i. $(result[1]) - Macro F1: $(round(result[3]*100, digits=2))%")
end

best_desc_tree, best_max_depth_tree, best_macro_f1_tree = sorted_tree_results[1][1:3]

println("\n✨ Best Decision Tree: $best_desc_tree")
println("   CV Macro F1: $(round(best_macro_f1_tree*100, digits=2))%")

# Train and evaluate final tree
println("\n🚀 Training final Decision Tree...")

normParams_tree = calculateMinMaxNormalizationParameters(Matrix{Float64}(train_inputs))
train_inputs_norm_tree = normalizeMinMax(Matrix{Float64}(train_inputs), normParams_tree)

if best_max_depth_tree == -1
    final_tree = DecisionTreeClassifier(min_samples_split=2)
else
    final_tree = DecisionTreeClassifier(max_depth=best_max_depth_tree, min_samples_split=2)
end

DecisionTree.fit!(final_tree, train_inputs_norm_tree, train_targets)

test_inputs_norm_tree = normalizeMinMax(Matrix{Float64}(test_inputs), normParams_tree)
tree_test_predictions = DecisionTree.predict(final_tree, test_inputs_norm_tree)

cm_tree, acc_tree, class_metrics_tree, macro_f1_tree, weighted_f1_tree = 
    confusionMatrixMulticlass(tree_test_predictions, test_targets, 3)

println("\n📊 DECISION TREE TEST RESULTS:")
displayConfusionMatrix(cm_tree)
println("\nMacro F1: $(round(macro_f1_tree*100, digits=2))% | Accuracy: $(round(acc_tree*100, digits=2))%")


🔬 EXPERIMENT 3: DECISION TREES
Testing 7 Configurations

[1/7] Testing: Depth=3
  Testing Decision Tree: max_depth=3, min_samples_split=2
    → Macro F1: 69.34% | Weighted F1: 69.32%

[2/7] Testing: Depth=5
  Testing Decision Tree: max_depth=5, min_samples_split=2
    → Macro F1: 76.56% | Weighted F1: 76.55%

[3/7] Testing: Depth=7
  Testing Decision Tree: max_depth=7, min_samples_split=2
    → Macro F1: 76.81% | Weighted F1: 76.79%

[4/7] Testing: Depth=10
  Testing Decision Tree: max_depth=10, min_samples_split=2
    → Macro F1: 76.36% | Weighted F1: 76.34%

[5/7] Testing: Depth=15
  Testing Decision Tree: max_depth=15, min_samples_split=2
    → Macro F1: 75.55% | Weighted F1: 75.54%

[6/7] Testing: Depth=20
  Testing Decision Tree: max_depth=20, min_samples_split=2
    → Macro F1: 74.62% | Weighted F1: 74.61%

[7/7] Testing: Depth=Unlimited
  Testing Decision Tree: max_depth=unlimited, min_samples_split=2
    → Macro F1: 71.21% | Weighted F1: 71.19%

🏆 Decision Tree Results Ranking

# Experiment 4: k-Nearest Neighbors (kNN)

## Overview:
- **Tested:** 6 different k values
- **Distance metric:** Euclidean
- **Algorithm:** KD-Tree for efficiency
- **Voting:** Majority voting among k neighbors

## k Values Tested:
1. k = 1 (most sensitive to noise)
2. k = 3
3. k = 5
4. k = 7
5. k = 10
6. k = 15 (most smooth decision boundary)

## Notes:
- **Normalization is CRITICAL** for kNN (distances must be comparable)
- No explicit training phase (lazy learner)
- Prediction can be slow for large datasets

In [12]:
# ============================================================================
#          EXPERIMENT 4: k-NEAREST NEIGHBORS (kNN)
# ============================================================================

println("\n" * "="^70)
println("🔬 EXPERIMENT 4: k-NEAREST NEIGHBORS")
println("Testing 6 k Values")
println("="^70)

k_values = [1, 3, 5, 7, 10, 15]

knn_results = []

for (i, k) in enumerate(k_values)
    println("\n[$i/6] Testing: k=$k")
    
    macro_f1, weighted_f1, cm, class_metrics = knnCrossValidation(
        (Matrix{Float64}(train_inputs), train_targets),
        cv_indices;
        k = k,
        verbose = true
    )
    
    push!(knn_results, (k, macro_f1, weighted_f1, cm, class_metrics))
end

sorted_knn_results = sort(knn_results, by=x->x[2], rev=true)

println("\n🏆 kNN Results Ranking:")
println("-"^70)
for (i, (k, macro_f1, weighted_f1, cm, class_metrics)) in enumerate(sorted_knn_results)
    badge = i == 1 ? "🥇" : i == 2 ? "🥈" : i == 3 ? "🥉" : "  "
    println("$badge $i. k=$k - Macro F1: $(round(macro_f1*100, digits=2))%")
end

best_k_knn, best_macro_f1_knn = sorted_knn_results[1][1:2]

println("\n✨ Best kNN: k=$best_k_knn")
println("   CV Macro F1: $(round(best_macro_f1_knn*100, digits=2))%")

# Prepare and evaluate final kNN
println("\n🚀 Preparing final kNN...")

normParams_knn = calculateMinMaxNormalizationParameters(Matrix{Float64}(train_inputs))
train_inputs_norm_knn = normalizeMinMax(Matrix{Float64}(train_inputs), normParams_knn)

test_inputs_norm_knn = normalizeMinMax(Matrix{Float64}(test_inputs), normParams_knn)
knn_test_predictions = knnPredict(train_inputs_norm_knn, train_targets, test_inputs_norm_knn, best_k_knn)

cm_knn, acc_knn, class_metrics_knn, macro_f1_knn, weighted_f1_knn = 
    confusionMatrixMulticlass(knn_test_predictions, test_targets, 3)

println("\n📊 kNN TEST RESULTS:")
displayConfusionMatrix(cm_knn)
println("\nMacro F1: $(round(macro_f1_knn*100, digits=2))% | Accuracy: $(round(acc_knn*100, digits=2))%")


🔬 EXPERIMENT 4: k-NEAREST NEIGHBORS
Testing 6 k Values

[1/6] Testing: k=1
  Testing kNN: k=1
    → Macro F1: 72.57% | Weighted F1: 72.56%

[2/6] Testing: k=3
  Testing kNN: k=3
    → Macro F1: 75.05% | Weighted F1: 75.04%

[3/6] Testing: k=5
  Testing kNN: k=5
    → Macro F1: 76.06% | Weighted F1: 76.05%

[4/6] Testing: k=7
  Testing kNN: k=7
    → Macro F1: 76.52% | Weighted F1: 76.51%

[5/6] Testing: k=10
  Testing kNN: k=10
    → Macro F1: 76.55% | Weighted F1: 76.54%

[6/6] Testing: k=15
  Testing kNN: k=15
    → Macro F1: 76.8% | Weighted F1: 76.78%

🏆 kNN Results Ranking:
----------------------------------------------------------------------
🥇 1. k=15 - Macro F1: 76.8%
🥈 2. k=10 - Macro F1: 76.55%
🥉 3. k=7 - Macro F1: 76.52%
   4. k=5 - Macro F1: 76.06%
   5. k=3 - Macro F1: 75.05%
   6. k=1 - Macro F1: 72.57%

✨ Best kNN: k=15
   CV Macro F1: 76.8%

🚀 Preparing final kNN...

📊 kNN TEST RESULTS:

📊 CONFUSION MATRIX (3x3):
                  Predicted
              Legit    Sospe

# Experiment 5: Ensemble Methods

## Overview:
Combining the **top 3 models** to improve robustness and performance.

### Models Selected:
1. **ANN** (best individual performer)
2. **Decision Tree** (interpretable, fast)
3. **kNN** (non-parametric, flexible)

### Ensemble Techniques:
1. **Majority Voting:** Each model votes, class with most votes wins
2. **Weighted Voting:** Models vote with weights proportional to their CV performance

### Expected Benefits:
- Reduce variance (model averaging)
- More robust predictions
- Leverage strengths of different algorithms

In [13]:
# ============================================================================
#            EXPERIMENT 5: ENSEMBLE METHODS
# ============================================================================

println("\n" * "="^70)
println("🔬 EXPERIMENT 5: ENSEMBLE METHODS")
println("Combining ANN + Decision Tree + kNN")
println("="^70)

all_predictions = [ann_test_predictions, tree_test_predictions, knn_test_predictions]

# Method 1: Majority Voting
println("\n[1/2] Majority Voting...")
majority_predictions = majorityVoting(all_predictions)

cm_majority, acc_majority, class_metrics_majority, macro_f1_majority, weighted_f1_majority = 
    confusionMatrixMulticlass(majority_predictions, test_targets, 3)

println("✅ Majority Voting: Macro F1 = $(round(macro_f1_majority*100, digits=2))%")

# Method 2: Weighted Voting
println("\n[2/2] Weighted Voting (based on CV performance)...")
cv_scores = [best_macro_f1_ann, best_macro_f1_tree, best_macro_f1_knn]
weights = cv_scores ./ sum(cv_scores)

println("  Model weights:")
println("    ANN:           $(round(weights[1]*100, digits=1))% (CV F1: $(round(best_macro_f1_ann*100, digits=2))%)")
println("    Decision Tree: $(round(weights[2]*100, digits=1))% (CV F1: $(round(best_macro_f1_tree*100, digits=2))%)")
println("    kNN:           $(round(weights[3]*100, digits=1))% (CV F1: $(round(best_macro_f1_knn*100, digits=2))%)")

weighted_predictions = weightedVoting(all_predictions, weights)

cm_weighted, acc_weighted, class_metrics_weighted, macro_f1_weighted, weighted_f1_weighted = 
    confusionMatrixMulticlass(weighted_predictions, test_targets, 3)

println("\n✅ Weighted Voting: Macro F1 = $(round(macro_f1_weighted*100, digits=2))%")

println("\n📊 ENSEMBLE (Weighted Voting) TEST RESULTS:")
displayConfusionMatrix(cm_weighted)
println("\nMacro F1: $(round(macro_f1_weighted*100, digits=2))% | Accuracy: $(round(acc_weighted*100, digits=2))%")


🔬 EXPERIMENT 5: ENSEMBLE METHODS
Combining ANN + Decision Tree + kNN

[1/2] Majority Voting...
✅ Majority Voting: Macro F1 = 77.45%

[2/2] Weighted Voting (based on CV performance)...
  Model weights:
    ANN:           33.5% (CV F1: 77.33%)
    Decision Tree: 33.3% (CV F1: 76.81%)
    kNN:           33.3% (CV F1: 76.8%)

✅ Weighted Voting: Macro F1 = 77.45%

📊 ENSEMBLE (Weighted Voting) TEST RESULTS:

📊 CONFUSION MATRIX (3x3):
                  Predicted
              Legit    Sospect   Fraud
----------------------------------------------------------------------
Legittimo    |    3849      298        0
Sospetto     |     636     2166     1290
Fraudolento  |       0      434     3720

Macro F1: 77.45% | Accuracy: 78.55%


# Final Results & Comparison

## Summary of All Experiments:

Comparing all 6 approaches (4 individual algorithms + 2 ensemble methods) on the **hold-out test set**.

### Evaluation Metrics:
- **Macro F1 Score:** Average F1 across all 3 classes (treats classes equally)
- **Accuracy:** Overall correct predictions
- **Per-Class Metrics:** Recall, Precision, F1 for each risk level

### Business Context:
- **Class 0 (Legittimo):** High precision preferred (avoid blocking legitimate transactions)
- **Class 1 (Sospetto):** Balance recall/precision (manual review capacity)
- **Class 2 (Fraudolento):** **High recall critical** (catch as many frauds as possible)

### Key Question:
**Which approach best balances overall performance with fraud detection rate?**

In [14]:
# ============================================================================
#              FINAL RESULTS COMPARISON
# ============================================================================

println("\n" * "="^70)
println("📊 FINAL RESULTS - ALL APPROACHES COMPARISON")
println("="^70)

println("\n🏆 PERFORMANCE RANKING (Test Set):")
println("="^70)
println("Rank | Approach                | Macro F1   | Accuracy   | Config")
println("-"^70)
println("1.   | ANN                     | $(rpad(round(macro_f1_ann*100, digits=2), 7))% | $(rpad(round(acc_ann*100, digits=2), 7))% | $best_topology_ann")
println("2.   | kNN                     | $(rpad(round(macro_f1_knn*100, digits=2), 7))% | $(rpad(round(acc_knn*100, digits=2), 7))% | k=$best_k_knn")
println("3.   | Decision Tree           | $(rpad(round(macro_f1_tree*100, digits=2), 7))% | $(rpad(round(acc_tree*100, digits=2), 7))% | depth=$best_max_depth_tree")
println("4.   | SVM                     | $(rpad(round(macro_f1_svm*100, digits=2), 7))% | $(rpad(round(acc_svm*100, digits=2), 7))% | $best_desc_svm")
println("-"^70)
println("5.   | Ensemble (Majority)     | $(rpad(round(macro_f1_majority*100, digits=2), 7))% | $(rpad(round(acc_majority*100, digits=2), 7))% | ANN+Tree+kNN")
println("6.   | Ensemble (Weighted)     | $(rpad(round(macro_f1_weighted*100, digits=2), 7))% | $(rpad(round(acc_weighted*100, digits=2), 7))% | ANN+Tree+kNN")
println("="^70)

# Determine overall best
all_scores = [
    ("ANN", macro_f1_ann),
    ("kNN", macro_f1_knn),
    ("Decision Tree", macro_f1_tree),
    ("SVM", macro_f1_svm),
    ("Ensemble Majority", macro_f1_majority),
    ("Ensemble Weighted", macro_f1_weighted)
]
best_overall = sort(all_scores, by=x->x[2], rev=true)[1]

println("\n🎯 BEST OVERALL APPROACH: $(best_overall[1])")
println("   Test Macro F1: $(round(best_overall[2]*100, digits=2))%")

# Fraud Detection Rate Comparison
println("\n💰 FRAUD DETECTION RATE (Class 2 - Recall):")
println("-"^70)
fraud_recall_ann = class_metrics_ann[3][2]
fraud_recall_tree = class_metrics_tree[3][2]
fraud_recall_knn = class_metrics_knn[3][2]
fraud_recall_svm = class_metrics_svm[3][2]
fraud_recall_weighted = class_metrics_weighted[3][2]

println("ANN:              $(round(fraud_recall_ann*100, digits=1))%")
println("Decision Tree:    $(round(fraud_recall_tree*100, digits=1))%")
println("kNN:              $(round(fraud_recall_knn*100, digits=1))%")
println("SVM:              $(round(fraud_recall_svm*100, digits=1))%")
println("Ensemble:         $(round(fraud_recall_weighted*100, digits=1))%")

println("\n" * "="^70)
println("✅ ALL EXPERIMENTS COMPLETE!")
println("="^70)
println("\n📋 SUMMARY:")
println("  ✅ Tested 4 ML algorithms (ANN, SVM, Decision Tree, kNN)")
println("  ✅ ANN: 8 topologies tested")
println("  ✅ SVM: 10 configurations tested")
println("  ✅ Decision Tree: 7 depths tested")
println("  ✅ kNN: 6 k values tested")
println("  ✅ Created 2 ensemble methods")
println("  ✅ All used proper Train/Test split (80/20)")
println("  ✅ 3-fold stratified cross-validation for parameter selection")
println("  ✅ Random seed (42) set for reproducibility")
println("  ✅ No data leakage (test set never seen during training)")
println("="^70)


📊 FINAL RESULTS - ALL APPROACHES COMPARISON

🏆 PERFORMANCE RANKING (Test Set):
Rank | Approach                | Macro F1   | Accuracy   | Config
----------------------------------------------------------------------
1.   | ANN                     | 77.86  % | 78.83  % | [256, 128]
2.   | kNN                     | 76.94  % | 77.41  % | k=15
3.   | Decision Tree           | 76.91  % | 78.18  % | depth=7
4.   | SVM                     | 74.87  % | 75.95  % | RBF C=10.0 γ=auto
----------------------------------------------------------------------
5.   | Ensemble (Majority)     | 77.45  % | 78.55  % | ANN+Tree+kNN
6.   | Ensemble (Weighted)     | 77.45  % | 78.55  % | ANN+Tree+kNN

🎯 BEST OVERALL APPROACH: ANN
   Test Macro F1: 77.86%

💰 FRAUD DETECTION RATE (Class 2 - Recall):
----------------------------------------------------------------------
ANN:              90.4%
Decision Tree:    89.0%
kNN:              76.6%
SVM:              85.7%
Ensemble:         89.6%

✅ ALL EXPERIMENTS COMPL